In [2]:
import time
import warnings
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, confusion_matrix

# 경고 메시지 무시
warnings.filterwarnings(action='ignore')

In [2]:
## --- 분석 수행 및 결과 저장 클래스 선언 ---
class CrackAnalyzer:
    
    def __init__(self, modeldir):
        self.modeldir = modeldir
        self.elapsed_time = 0
        self.predict_list = []
        
    def analysis(self, input_list):
        
        with tf.Session() as sess:
            print("START : " + self.modeldir)
            
            # 모델 로딩
            tf.saved_model.loader.load(sess, ["serve"], self.modeldir)
            
            graph = tf.get_default_graph()
            
            x = graph.get_tensor_by_name("in:0")
            out = graph.get_tensor_by_name("pred:0")

            print('input_list : {}'.format(input_list.shape))
            print('out : {}'.format(out))
            
            # 모델 분석 수행
            start_time = time.time()
            _out = sess.run(out, feed_dict={x:input_list})
            end_time = time.time()
            
            self.elapsed_time = end_time - start_time            
            self.predict_list = _out
            
#             for index in range(0, input_list.shape[0]):
#                 self.predict_list.append(np.argmax(_out[0][index], axis=0))
                
            print("END : " + self.modeldir)
            
            return self.predict_list

In [3]:
### 설정 변수 정의 ###
# 데이터 구조
# train_list : 학습용 데이터(이미지 개수, 448, 448, 1)
# label_list : 학습용 라벨 데이터(이미지 개수, 448, 448, 1)

# 학습 데이터 파일명
train_list_file = "./images.npy"
# 라벨 데이터 파일명
label_list_file = "./masks.npy"

print("load train data: ", train_list_file) 
train_list = np.load(train_list_file, allow_pickle=True)
print("load complete")

print("load train label data: ", label_list_file)
label_list = np.load(label_list_file, allow_pickle=True)
# label_list = label_list[:,2]
# label_list = label_list[:,2]
print("load complete")

print(train_list.shape)
print(label_list.shape)

load train data:  ./images.npy
load complete
load train label data:  ./masks.npy
load complete
(11298, 448, 448, 1)
(11298, 448, 448, 1)


In [4]:
## --- 분석 데이터에서 분석 수행용 데이터 일부 추출 ---
# 분석 데이터 설정
# 분석할 데이터 개수
analysis_count = 100
# 분석 데이터 중 최초 데이터 위치 
#base_index = np.random.randint(label_list.shape[0] - analysis_count)
base_index = 0

# 분석용 데이터와 라벨
analysis_x = train_list[base_index:base_index + analysis_count]
analysis_y = label_list[base_index:base_index + analysis_count]
print('analysis_x : {}'.format(analysis_x.shape))

analysis_x : (100, 448, 448, 1)


In [5]:
## --- 분석 모델 로딩 및 분석 수행 ---
# 분석 모델 디렉토리명
modeldir = "/src/hyebin/model/UNET/CRACK-UNET-24"

analyzer = CrackAnalyzer(modeldir)
predict_list = analyzer.analysis(analysis_x)
print(predict_list.shape)

START : /src/hyebin/model/UNET/CRACK-UNET-24
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from /src/hyebin/model/UNET/CRACK-UNET-24/variables/variables
input_list : (100, 448, 448, 1)
out : Tensor("pred:0", shape=(?, 448, 448, 1), dtype=float32)
END : /src/hyebin/model/UNET/CRACK-UNET-24
(100, 448, 448, 1)


In [6]:
### INPUT ###
## 예측값, 라벨값, Threshold
threshold = 0.1

pred_list = []
label_list = []

for index in range(analysis_count):
    # 예측값 
    predict = analyzer.predict_list[index]
    pred = np.where(predict < threshold, 1, 0)

    # 라벨값
    label = analysis_y[index]
    
    pred_list.append(pred)
    label_list.append(label)

In [7]:
### 1. 정밀도(오탐) 함수 ###

label = np.array(label_list)
print(label.shape)
pred = np.array(pred_list)
print(pred.shape)

accuracy = np.mean(np.equal(label,pred))
print('accuracy  : ', accuracy)
right = np.sum(label * pred == 1) # ==1 때문에 1이 나옴.. 
print('right     : ', right)
precision = right / np.sum(pred)
print('precision : ', precision)
print('np.sum(pred)    : ', np.sum(pred))
recall = right / np.sum(label)
print('np.sum(label)    : ', np.sum(label))
print('recall    : ', recall)
f1 = 2 * precision*recall/(precision+recall)
print('f1        : ', f1)

# 라벨
label = np.squeeze(label, axis=3)
#label = label[:,-1]
label = label.reshape(-1)  # 1차원으로 바꾸기
print(label.shape)
#print(label)

# 예측
pred = np.squeeze(pred, axis=3)
pred = pred.reshape(-1)
print(pred.shape)

# accuracy: (tp + tn) / (p + n)
print('\naccuracy: %f'%accuracy_score(label,pred))
# precision: tp / (tp + fp)
print('precision: %f'%precision_score(label,pred))  # binary(이진법)가 되야함
# recall: tp / (tp + fn)
# f1_score
print('recall: %f'%recall_score(label,pred))
print('f1_score: %f'%f1_score(label,pred))

print('\n')
print(classification_report(label,pred,target_names=['class 0', 'class 1']))

(100, 448, 448, 1)
(100, 448, 448, 1)
accuracy  :  0.13883520009566327
right     :  7771
precision :  0.0004580488063941279
np.sum(pred)    :  16965441
np.sum(label)    :  334023.0
recall    :  0.02326486499432674
f1        :  0.0008984093380002987
(20070400,)
(20070400,)

accuracy: 0.138835
precision: 0.000458
recall: 0.023265
f1_score: 0.000898


              precision    recall  f1-score   support

     class 0       0.89      0.14      0.24  19736377
     class 1       0.00      0.02      0.00    334023

    accuracy                           0.14  20070400
   macro avg       0.45      0.08      0.12  20070400
weighted avg       0.88      0.14      0.24  20070400



In [5]:
### 2. 재현율(미탐) 함수 ###
labels = np.array([1,1,1,1,0,0]) #0은 정상(실력자), 1은 음치
guesses = np.array([1,1,0,0,0,0]) #나의 예측

accuracy = np.mean(np.equal(labels,guesses))
right = np.sum(labels * guesses == 1)
print('right:',right)
precision = right / np.sum(guesses)
print('np.sum(guesses):',np.sum(guesses))
recall = right / np.sum(labels)
print('np.sum(labels):',np.sum(labels))
f1 = 2 * precision*recall/(precision+recall)

print('accuracy',accuracy)
print('precision', precision)
print('recall', recall)
print('f1', f1)

print('\naccuracy: ', accuracy_score(labels, guesses))	# 0.3
print('precision: ', precision_score(labels, guesses))	# 0.5
print('recall: ', recall_score(labels, guesses))	# 0.42
print('f1_score: ', f1_score(labels, guesses))	# 0.46

print('\n')
print(classification_report(label,pred,target_names=['class 0', 'class 1']))

right: 2
np.sum(guesses): 2
np.sum(labels): 4
accuracy 0.6666666666666666
precision 1.0
recall 0.5
f1 0.6666666666666666

accuracy:  0.6666666666666666
precision:  1.0
recall:  0.5
f1_score:  0.6666666666666666




NameError: name 'label' is not defined

In [9]:
### 3. 정확도 함수 ###